In [1]:
from random import seed, randint
import ast; import sys; from time import process_time
import numpy as np; import verbose_module
from pickle import load, dump
from copy import deepcopy

sys.path.append('../../.')
import pIRPgym

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

########################     Instance generator and Environment     #########################
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']; look_ahead = ['d','q']

# Historical data
historical_data = ['*']

# Other parameters
backorders = 'backorders'

# Random Instance
q_params = {'dist': 'c_uniform', 'r_f_params': [110,130]}                                         # Supply
p_params = {'dist': 'd_uniform', 'r_f_params': [80,120]}                                          # Purchase costs at suppliers'

d_params = {'dist': 'log-normal-trunc', 'r_f_params': [3,1], 'break':104.5, 'offset':15}          # Demand
h_params = {'dist': 'd_uniform', 'r_f_params': [20,40]}                                           # Holding costs

### Environment 
# Creating environment object
inventory = True; routing = True; perishability = "ages"
env = pIRPgym.steroid_IRP(routing, inventory, perishability)

# Experiments

In [2]:
T = 30; verbose = False; agg = False; hold_cost = True; save = True
rr = 0.1; thetas = [0.6,0.7,0.8,0.9]; gamma = 1

weights = [[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0]]
objectives = ["costs","climate","water","land","fossil","acid","eutroph","ozone"]

file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "rb")
feas_seeds = load(file); file.close()

In [3]:
for theta in thetas:
    
    seed_ix = 0
    env_config = {'M':5,'K':3,'T':T,'F':2,'Q':400,'S':5,'LA_horizon':7,'d_max':2000,'hist_window':100, 'theta':theta, 'hold_cost':hold_cost, 'rr':rr, 'gamma':gamma}

    for stoch_seed in feas_seeds:
        
        try:

            for w in weights:

                rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()

                stoch_rd_seed = stoch_seed; det_rd_seed = 350 # Random seeds
                obj_weights = dict(zip(objectives,w))
                metric = "".join([k*v for k,v in obj_weights.items()])

                # Creating instance generator object
                inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
                inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

                # Resetting the environment
                states[1] = env.reset(inst_gen, return_state=True)

                done = False; day = 1
                while not done:

                    ''' Inventory Routing Problem '''
                    m, variables, costs, impacts = pIRPgym.Inventory.IRP.build_model(states[day], env, inst_gen)

                    if metric == "costs":
                        action, la_decisions[day] = pIRPgym.Inventory.optimize_costs("land", m, costs, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
                        states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=False)
                    else:
                        action, la_decisions[day] = pIRPgym.Inventory.optimize_environmental_indicator(metric, m, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
                        states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=True)
                    
                    day += 1
                
                if save:
                        other_path = True
                        verbose_module.export_results.export_instance_parameters(w, seed_ix, inst_gen, other_path, theta=theta, gamma=gamma, min_q=rr)
                        verbose_module.export_results.export_actions(w, seed_ix, actions, other_path, theta=theta, gamma=gamma, min_q=rr)
                        verbose_module.export_results.export_lookahead_decisions(w, seed_ix, la_decisions, other_path, theta=theta, gamma=gamma, min_q=rr)
                        verbose_module.export_results.export_inventory(w, seed_ix, states, other_path, theta=theta, gamma=gamma, min_q=rr)
                        verbose_module.export_results.export_rewards(w, seed_ix, rewards, other_path, theta=theta, gamma=gamma, min_q=rr)
                        verbose_module.export_results.export_backorders(w, seed_ix, backo, other_path, theta=theta, gamma=gamma, min_q=rr)
                        verbose_module.export_results.export_perished(w, seed_ix, perished, other_path, theta=theta, gamma=gamma, min_q=rr)
                        verbose_module.export_results.export_norm_matrix(w, seed_ix, norm_matrix, other_path, theta=theta, gamma=gamma, min_q=rr)
                    
                print(f"\t{w}")
            print(f"Done seed number {seed_ix} - {stoch_seed}")

        except:
            
            print(f"FAILED SEED NUMBER {seed_ix} - {stoch_seed}")

        seed_ix += 1

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
	[0, 1, 0, 0, 0, 0, 0, 0]
FAILED SEED NUMBER 0 - 122713075


# Verifying that the env. indicators don't matter to correct the costs action

In [ ]:
T = 7; verbose = False; agg = False; hold_cost = True; save = True
rr = 0.1; theta = 0.95; gamma = 1

env_config = {'M':5,'K':2,'T':T,'F':2,'Q':375,'S':3,'LA_horizon':7,'d_max':2000,'hist_window':80, 'theta':theta, 'hold_cost':hold_cost, 'rr':rr, 'gamma':gamma}

file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "rb")
feas_seeds = load(file); file.close()

In [ ]:
seed_ix = 0
for stoch_seed in feas_seeds:
    print(f"Seed {seed_ix}")
    
    for metric in ["climate","water","land","fossil"]:

        rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()

        stoch_rd_seed = stoch_seed; det_rd_seed = 350 # Random seeds

        # Creating instance generator object
        inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
        inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

        # Resetting the environment
        states[1] = env.reset(inst_gen, return_state=True)

        done = False; day = 1
        print(f"\tCorrecting for {metric}")
        try:
            while not done:

                ''' Inventory Routing Problem '''
                m, variables, costs, impacts = pIRPgym.Inventory.IRP.build_model(states[day], env, inst_gen)
                
                action, la_decisions[day] = pIRPgym.Inventory.optimize_costs(metric, m, costs, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
                states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=True)

                day += 1
        except:
            continue
    seed_ix += 1

Seed 0
	Correcting for climate
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
			(1.664, 1.5381)	(0.1834, 0.1702)	(14.3924, 13.1065)	(28.5068, 26.5835)
			(1.3957, 1.3711)	(0.164, 0.1615)	(9.2351, 8.984)	(27.2776, 26.902)
			(1.3053, 1.3036)	(0.1414, 0.1412)	(11.9732, 11.9559)	(21.5515, 21.5256)
			(1.623, 1.5221)	(0.1729, 0.1623)	(15.715, 14.6849)	(25.8138, 24.2731)
			(1.1385, 1.118)	(0.1329, 0.1308)	(7.7849, 7.575)	(21.9532, 21.6393)
			(1.2875, 1.0676)	(0.16, 0.137)	(6.0949, 3.849)	(28.042, 24.6828)
			(0.1366, 0.1366)	(0.0158, 0.0158)	(0.9668, 0.9668)	(2.5946, 2.5946)
	Correcting for water
			(1.664, 1.5381)	(0.1834, 0.1702)	(14.3924, 13.1065)	(28.5068, 26.5835)
			(1.3957, 1.3711)	(0.164, 0.1615)	(9.2351, 8.984)	(27.2776, 26.902)
			(1.3053, 1.3036)	(0.1414, 0.1412)	(11.9732, 11.9559)	(21.5515, 21.5256)
			(1.5384, 1.5221)	(0.164, 0.1623)	(14.8513, 14.6849)	(24.522, 24.2731)
			(1.1385, 1.118)	(0.1329, 0.1308)	(7.7849, 7.575)	(21.9532, 

# Random seeds search

In [2]:
T = 30; verbose = False; agg = False; hold_cost = True; save = True
rr = 0.1; theta = 0.95; gamma = 1

env_config = {'M':5,'K':3,'T':T,'F':2,'Q':375,'S':5,'LA_horizon':7,'d_max':2000,'hist_window':100, 'theta':theta, 'hold_cost':hold_cost, 'rr':rr, 'gamma':gamma}
weights = [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]
objectives = ["costs","climate","water","land","fossil"]

file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "rb")
feas_seeds = load(file); file.close()

In [3]:
feas_seeds = list()
seeds = [randint(120, 156789276) for ix in range(4000)]

seed_ix = 0
while len(feas_seeds) < 50:

    try:
        for w in weights:

            rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()

            stoch_rd_seed = seeds[seed_ix]; det_rd_seed = 350 # Random seeds
            obj_weights = dict(zip(objectives,w))
            metric = "".join([k*v for k,v in obj_weights.items()])

            # Creating instance generator object
            inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
            inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

            # Resetting the environment
            states[1] = env.reset(inst_gen, return_state=True)

            done = False; day = 1
            while not done:

                ''' Inventory Routing Problem '''
                m, variables, costs, impacts = pIRPgym.Inventory.IRP.build_model(states[day], env, inst_gen)

                if metric == "costs":
                    action, la_decisions[day] = pIRPgym.Inventory.optimize_costs("land", m, costs, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
                    states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=True)
                else:
                    action, la_decisions[day] = pIRPgym.Inventory.optimize_environmental_indicator(metric, m, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
                    states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=False)
                
                day += 1
        
        print(f"FOUND FEASIBLE SEED NUMBER {len(feas_seeds)} - {seeds[seed_ix]}")
        feas_seeds.append(seeds[seed_ix])

    except:

        print(f"Failed seed at attempt {seed_ix+1} {seeds[seed_ix]}")
    
    seed_ix += 1

file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "wb")
dump(feas_seeds, file); file.close()


Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
FOUND FEASIBLE SEED NUMBER 0 - 122713075
Failed seed at attempt 2 90448019
FOUND FEASIBLE SEED NUMBER 1 - 72711823
FOUND FEASIBLE SEED NUMBER 2 - 77416148
FOUND FEASIBLE SEED NUMBER 3 - 44082509
FOUND FEASIBLE SEED NUMBER 4 - 6920436
FOUND FEASIBLE SEED NUMBER 5 - 62168654
FOUND FEASIBLE SEED NUMBER 6 - 37772543
FOUND FEASIBLE SEED NUMBER 7 - 80165988
FOUND FEASIBLE SEED NUMBER 8 - 137433499
FOUND FEASIBLE SEED NUMBER 9 - 60800317
FOUND FEASIBLE SEED NUMBER 10 - 118039527
FOUND FEASIBLE SEED NUMBER 11 - 16646696
Failed seed at attempt 14 59166266
FOUND FEASIBLE SEED NUMBER 12 - 36672622
FOUND FEASIBLE SEED NUMBER 13 - 5415991
FOUND FEASIBLE SEED NUMBER 14 - 67782058
FOUND FEASIBLE SEED NUMBER 15 - 41708477
FOUND FEASIBLE SEED NUMBER 16 - 15668578
FOUND FEASIBLE SEED NUMBER 17 - 47815753
FOUND FEASIBLE SEED NUMBER 18 - 132465846
FOUND FEASIBLE SEED NUMBER 19 - 130612941
FOUND FEASIBLE SEED NUMBER 